# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

![heatmap](images/heatmap.png)

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps


# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_csv = "output_data/cities.csv"
city_data = pd.read_csv(city_csv)
city_data.describe()


,City ID,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
count,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,5.680000e+02
mean,283.500000,18.567779,21.410554,51.691549,73.375000,56.042254,7.694507,1.613104e+09
std,164.111751,33.377770,91.540692,32.616273,18.614872,39.511769,6.010969,2.884660e+02
min,0.000000,-54.800000,-175.200000,-40.000000,17.000000,0.000000,0.220000,1.613104e+09
25%,141.750000,-8.689375,-57.830675,29.600000,64.000000,13.750000,3.440000,1.613104e+09
50%,283.500000,21.283300,25.468400,64.275000,77.000000,75.000000,6.085000,1.613104e+09
75%,425.250000,46.572525,105.734975,76.515000,88.000000,93.000000,10.360000,1.613104e+09
max,567.000000,78.218600,178.416700,95.000000,100.000000,100.000000,47.180000,1.613104e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
fig = gmaps.figure()

In [4]:
humidity_data = city_data[["Lat", "Lng","Humidity"]]
hm_locations = humidity_data[["Lat", "Lng"]].astype(float)
weights = humidity_data["Humidity"].astype(float)

In [5]:
heat_layer = gmaps.heatmap_layer(hm_locations, 
                                 weights=weights, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3
                                )

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows with null values.

In [6]:
ideal_conditions = city_data.loc[(city_data["Max Temp"] >= 70) & 
                                 (city_data["Max Temp"] < 80) & 
                                 (city_data["Wind Speed"] < 10) & 
                                 (city_data["Cloudiness"] == 0),:
                                ]
if len(ideal_conditions) > 10:
    ideal_conditions = ideal_conditions.head(10)

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = pd.DataFrame(ideal_conditions)

hotel_df["Hotel Name"] = ""

hotels = []

for index, city in hotel_df.iterrows():
    
    hotel_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    lat = city['Lat']
    long = city['Lng']
    
    params = {
        "location": f'{lat},{long}',
        "keyword": 'hotel',
        "radius": 5000,
        "key": g_key
    }

    response = requests.get(hotel_url, params=params).json()
    try:
        hotels.append(response['results'][0]['name'])
    except:
        hotels.append("")
    
hotel_df["Hotel Name"] = hotels
hotel_df = hotel_df.loc[hotel_df["Hotel Name"]!="",:]
hotel_df

,City ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
4,4,victoria,22.2855,114.1577,71.01,56,0,5.99,HK,1613103554,The OTTO Hotel
15,15,atuona,-9.8000,-139.0333,79.66,74,0,9.91,PF,1613103560,Hôtel Hiva Oa Hanakee Lodge
81,81,sao filipe,14.8961,-24.4956,71.46,66,0,9.91,CV,1613103870,Hotel Xaguate
108,108,pasni,25.2631,63.4710,72.90,46,0,3.56,PK,1613103830,Juddi Hotel
133,133,along,28.1667,94.7667,77.36,27,0,1.19,IN,1613104362,Hotel West
142,142,busselton,-33.6500,115.3333,73.08,41,0,1.97,AU,1613103821,The Sebel Busselton
156,156,ulladulla,-35.3500,150.4667,78.01,65,0,9.22,AU,1613103645,Beachhouse Mollymook
233,233,falam,22.9167,93.6833,73.13,21,0,2.89,MM,1613104376,Guesthouse of Government Administration Depart...
260,260,hopelchen,19.7667,-89.8500,74.71,74,0,6.53,MX,1613104380,OYO Hotel Los Arcos


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))